##### This notebook demonstrates a demo of how to deploy services, jobs and python functions on truefoundry platform.

## Try this Notebook in Google Colab

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/truefoundry/truefoundry-examples/blob/main/deployment-quickstart/truefoundry_deployment_quickstart.ipynb)

# 🛠 Setup

In [ ]:
pip install -U "servicefoundry==0.6.6" "mlfoundry==0.6.1"

In [1]:
import logging
[logging.root.removeHandler(h) for h in logging.root.handlers]
logging.basicConfig(level=logging.INFO, format='%(asctime)s [%(name)s] %(levelname)-8s %(message)s')

In [ ]:
!sfy login

In [ ]:
# Copy workspace FQN from https://app.truefoundry.com/workspaces
from getpass import getpass

WORKSPACE="tfy-cluster-euwe1:np18-ws"
TFY_API_KEY = getpass("Please enter your API Key,( https://app.truefoundry.com/settings ):-")

In [ ]:
# clone the latest code
!git clone https://github.com/truefoundry/truefoundry-examples
%cd truefoundry-examples/deployment-quickstart/

# ⚡ Jobs - Quick Start
Here we will create a simple job that counts to 30 sleeping for 1 second every iteration.

In [ ]:
%cd job/

### Defining a simple job

In [5]:
!pygmentize run.py

import argparse
import time


def main(limit: int):
    for i in range(limit):
        print(f"Running Job: {i + 1}")
        time.sleep(1)

if __name__ == '__main__':
    parser = argparse.ArgumentParser()
    parser.add_argument("--limit", default=10, type=int)
    args = parser.parse_args()
    main(args.limit)


### Deploying the job using truefoundry

In [ ]:
import logging

from servicefoundry import Build, Job, PythonBuild, Resources

logging.basicConfig(level=logging.INFO, format='%(asctime)s %(name)-12s %(levelname)-8s %(message)s', force=True)

job = Job(
    name="counter-job",
    image=Build(build_spec=PythonBuild(command="python run.py")),
    resources=Resources(cpu_request=0.5, cpu_limit=1.0)
)
job.deploy(workspace_fqn=WORKSPACE)

# ☁ Service - Quick Start
Here we will take a simple fastapi service and deploy it using truefoundry python-sdk

In [ ]:
%cd ../service/

### Defining simple fastapi service

In [7]:
!pygmentize service.py

from fastapi import FastAPI

app = FastAPI()


@app.get("/")
async def root():
    return {"message": "Hello World"}


### Defining the requirements

In [8]:
!pygmentize requirements.txt

fastapi==0.89.1
uvicorn==0.20.0


### Deploying the fastapi service

In [ ]:
import logging

from servicefoundry import Build, Service, PythonBuild, Resources

logging.basicConfig(level=logging.INFO, format='%(asctime)s %(name)-12s %(levelname)-8s %(message)s', force=True)

service = Service(
    name="fastapi-service",
    image=Build(build_spec=PythonBuild(command="uvicorn service:app --host 0.0.0.0 --port 8080")),
    resources=Resources(cpu_request=0.5, cpu_limit=1.0),
    ports=[{"port": 8080}]
)
service.deploy(workspace_fqn=WORKSPACE)

# ☁ Function as Service - Quick Start
Here we will take two python functions that use numpy and register them as routes in a deployed web service

In [9]:
%cd ../function-service/

/Users/nikp18/work/repos/tempp/truefoundry-examples/deployment-quickstart/truefoundry-examples/deployment-quickstart/function-service


### Defining some basic functions

In [11]:
!pygmentize functions.py

from typing import List
import numpy as np


def multiply(a: float, b: float):
  return a * b 


def normal(loc: float, scale: float, size: List[int]):
    return np.random.normal(loc=loc, scale=scale, size=size).tolist()


def uniform(low: float, high: float, size: List[int]):
    return np.random.uniform(low=low, high=high, size=size).tolist()


### Defining the requirements

In [12]:
!pygmentize requirements.txt

numpy==1.21.6


### Deploying the functions

In [ ]:
import logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s %(name)-12s %(levelname)-8s %(message)s', force=True)

from functions import normal, uniform, multiply
from servicefoundry.function_service import FunctionService

service = FunctionService(name="fn-service")
service.register_function(normal)
service.register_function(uniform)
service.register_function(multiply)
deployment = service.deploy(workspace_fqn=WORKSPACE)